<a href="https://colab.research.google.com/github/txc-000/repo-3/blob/main/percobaan6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import files
uploaded = files.upload()

Saving bbc_news.csv to bbc_news.csv


In [1]:
import pandas as pd
import tensorflow as tf

# Path ke file langsung
file_path = 'bbc_news.csv'

# Baca CSV dan ambil kolom title sebagai headline
headlines_df = pd.read_csv(file_path)
all_headlines = list(headlines_df.title.values)

print(f"Jumlah headline: {len(all_headlines)}")


Jumlah headline: 42115


In [2]:
print(all_headlines[:20])


['Ukraine: Angry Zelensky vows to punish Russian atrocities', 'War in Ukraine: Taking cover in a town under attack', "Ukraine war 'catastrophic for global food'", "Manchester Arena bombing: Saffie Roussos's parents on hearing the truth", 'Ukraine conflict: Oil price soars to highest level since 2008', 'Ukraine war: PM to hold talks with world leaders on further sanctions', 'Ukraine war: UK grants 50 Ukrainian refugee visas so far', 'TikTok limits services as Netflix pulls out of Russia', "Covid: Fourth jab for Scotland's vulnerable, and testing wind down fears in Wales", 'Protests across Russia see thousands detained', 'Ukraine conflict: Your guide to understanding day 11', 'Russian gymnast investigated for wearing pro-war symbol on podium next to Ukrainian', 'Ukraine: With placards and tears, Poles are greeting refugees like family', 'Twitter is part of our war effort - Ukraine minister', "Ukraine invasion: Volunteers 'working on autopilot'", 'Ukraine crisis: The West fights back agai

In [3]:
all_headlines[:20]

['Ukraine: Angry Zelensky vows to punish Russian atrocities',
 'War in Ukraine: Taking cover in a town under attack',
 "Ukraine war 'catastrophic for global food'",
 "Manchester Arena bombing: Saffie Roussos's parents on hearing the truth",
 'Ukraine conflict: Oil price soars to highest level since 2008',
 'Ukraine war: PM to hold talks with world leaders on further sanctions',
 'Ukraine war: UK grants 50 Ukrainian refugee visas so far',
 'TikTok limits services as Netflix pulls out of Russia',
 "Covid: Fourth jab for Scotland's vulnerable, and testing wind down fears in Wales",
 'Protests across Russia see thousands detained',
 'Ukraine conflict: Your guide to understanding day 11',
 'Russian gymnast investigated for wearing pro-war symbol on podium next to Ukrainian',
 'Ukraine: With placards and tears, Poles are greeting refugees like family',
 'Twitter is part of our war effort - Ukraine minister',
 "Ukraine invasion: Volunteers 'working on autopilot'",
 'Ukraine crisis: The West f

In [4]:
# Remove all headlines with the value of "Unknown"
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

42115

In [5]:
all_headlines[:20]

['Ukraine: Angry Zelensky vows to punish Russian atrocities',
 'War in Ukraine: Taking cover in a town under attack',
 "Ukraine war 'catastrophic for global food'",
 "Manchester Arena bombing: Saffie Roussos's parents on hearing the truth",
 'Ukraine conflict: Oil price soars to highest level since 2008',
 'Ukraine war: PM to hold talks with world leaders on further sanctions',
 'Ukraine war: UK grants 50 Ukrainian refugee visas so far',
 'TikTok limits services as Netflix pulls out of Russia',
 "Covid: Fourth jab for Scotland's vulnerable, and testing wind down fears in Wales",
 'Protests across Russia see thousands detained',
 'Ukraine conflict: Your guide to understanding day 11',
 'Russian gymnast investigated for wearing pro-war symbol on podium next to Ukrainian',
 'Ukraine: With placards and tears, Poles are greeting refugees like family',
 'Twitter is part of our war effort - Ukraine minister',
 "Ukraine invasion: Volunteers 'working on autopilot'",
 'Ukraine crisis: The West f

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None
)

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the words in our headlines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words)

Total words:  32637


In [8]:
# Print a subset of the word_index dictionary created by Tokenizer
subset_dict = {key: value for key, value in tokenizer.word_index.items() \
               if key in ['a','man','a','plan','a','canal','panama']}
print(subset_dict)

{'a': 8, 'man': 31, 'plan': 192, 'canal': 3979, 'panama': 8220}


In [9]:
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])

[[8], [31], [8], [192], [8], [3979], [8220]]

In [10]:
# Convert data to sequence of tokens
input_sequences = []
for line in all_headlines:
    # Convert our headline into a sequence of tokens
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Create a series of sequences for each headline
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:5]

['ukraine angry', 'ukraine angry zelensky', 'ukraine angry zelensky vows', 'ukraine angry zelensky vows to', 'ukraine angry zelensky vows to punish']


[[19, 1714],
 [19, 1714, 499],
 [19, 1714, 499, 662],
 [19, 1714, 499, 662, 1],
 [19, 1714, 499, 662, 1, 6103]]

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences with zeros at the beginning to make them all max length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         19, 1714], dtype=int32)

In [12]:
# Predictors are every word except the last
predictors = input_sequences[:,:-1]
# Labels are the last word
labels = input_sequences[:,-1]
labels[:5]

array([1714,  499,  662,    1, 6103], dtype=int32)

In [13]:
from tensorflow.keras import utils

labels = utils.to_categorical(labels, num_classes=total_words)

In [14]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Input is max sequence length - 1, as we've removed the last word for the label
input_len = max_sequence_len - 1

model = Sequential()

# Add input embedding layer
model.add(Embedding(total_words, 10, input_length=input_len))

# Add LSTM layer with 100 units
model.add(LSTM(100))
model.add(Dropout(0.1))

# Add output layer
model.add(Dense(total_words, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
import numpy as np

dummy_predictors = np.random.rand(100, predictors.shape[1]).astype(np.float32)
dummy_labels = np.random.rand(100, labels.shape[1]).astype(np.float32)

model.fit(dummy_predictors, dummy_labels, epochs=3, batch_size=8)


Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 107ms/step - loss: 169683.4531
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - loss: 169652.9531
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - loss: 169590.3125


In [18]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print(f"Total RAM: {ram_gb:.2f} GB")


Total RAM: 13.61 GB


In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
import gc

# Bersihkan memori
K.clear_session()
gc.collect()

# Contoh data dummy
num_samples = 10000
num_features = 10
num_classes = 3

predictors = np.random.rand(num_samples, num_features).astype(np.float32)
labels = np.random.randint(0, num_classes, size=(num_samples,))
labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)

# Buat dataset tf.data
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices((predictors, labels))
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Model sederhana
model = Sequential([
    Dense(64, activation='relu', input_shape=(num_features,)),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Training dengan dataset
model.fit(dataset, epochs=10, verbose=1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,883 (11.26 KB)

 Trainable params: 2,883 (11.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3416 - loss: 1.1007
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3439 - loss: 1.0993
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3262 - loss: 1.0989
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3420 - loss: 1.0993
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3355 - loss: 1.0986
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3480 - loss: 1.0976
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3493 - loss: 1.0974
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3423 - loss: 1.0971
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.3440 - loss: 1.0970
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3589 - loss: 1.0963


In [20]:
import numpy as np

def predict_next_token(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_class = np.argmax(predicted_probs, axis=-1)
    return predicted_class[0]  # hasil berupa 1 angka token ID

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Contoh data: kumpulan kalimat untuk tokenizer
texts = ["today in new york", "hello world", "machine learning is fun"]

# Buat tokenizer dan fit di data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Ubah kalimat input jadi sequence token angka
sequence = tokenizer.texts_to_sequences(["today in new york"])

# Padding/truncating supaya panjang sequence jadi 10 (sesuai input model)
padded_seq = pad_sequences(sequence, maxlen=10, padding='pre')

print(padded_seq.shape)  # (1, 10)

# Lalu inputkan ke model
prediction = model.predict(padded_seq)
print(prediction)


(1, 10)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
[[0.4925996  0.27832222 0.22907811]]


In [26]:
prediction = model.predict(padded_seq)  # shape (1, num_classes)
predicted_index = prediction.argmax(axis=-1)[0]  # ambil kelas dengan prediksi tertinggi
predicted_word = tokenizer.index_word.get(predicted_index, "unknown")
print(predicted_word)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
unknown


In [27]:
def generate_headline(seed_text, next_words=1):
    for _ in range(next_words):
        # Predict next token
        prediction = predict_next_token(seed_text)
        # Convert token to word
        next_word = tokenizer.sequences_to_texts([[prediction]])[0]

        # Add next word to the headline. This headline will be used in the next pass of the loop.
        seed_text += " " + next_word
    # Return headline as title-case
    return seed_text.title()

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_headline(seed_text, next_words=5):
    for _ in range(next_words):
        # Tokenize seed_text jadi sequence angka
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # Padding/truncating jadi panjang 10 sesuai model input
        token_list = pad_sequences([token_list], maxlen=10, padding='pre', truncating='pre')
        # Prediksi output (misal distribusi kelas kata)
        predicted_probs = model.predict(token_list, verbose=0)
        # Ambil kelas dengan probabilitas tertinggi
        predicted_index = np.argmax(predicted_probs, axis=-1)[0]
        # Cari kata dari tokenizer index_word
        output_word = tokenizer.index_word.get(predicted_index, "")
        # Tambah kata hasil prediksi ke seed_text
        seed_text += " " + output_word
    return seed_text


In [31]:
seed_texts = [
    'washington dc is',
    'today in new york',
    'the school district has',
    'crime has become']

for seed in seed_texts:
    print(generate_headline(seed, next_words=5))


washington dc is today today   
today in new york     
the school district has today today today  
crime has become today today today  


In [32]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}